In [1]:
import pandas as pd
import tiktoken
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import faiss
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [2]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaForCausalLM
from peft import PeftModel, PeftConfig
import torch

/home/crysis/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import dotenv_values
from langchain.llms import HuggingFacePipeline
import os

env_vars = dotenv_values('../.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = env_vars.get('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = env_vars.get('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = env_vars.get('LANGCHAIN_API_KEY')

In [4]:
df = pd.read_csv('./data/law-all-data.csv')
df = df.drop(['input'], axis = 1)
df

,instruction,output
0,동거 중인데 갑자기 헤어지자는 통보를 받았어요. 사실혼인 경우에도 위자료를 받을 수...,위자료를 받을 수 있습니다. 사실혼은 부부간 합의 또는 부부 어느 한 쪽의 일방적인...
1,이혼이 취소될 수 있나요?,"재판상 이혼은 재판절차를 거쳐 이혼판결이 선고된 것이므로 취소될 수 없지만, 협의이..."
2,이혼해도 자녀를 만날 수 있나요?,이혼 후 자녀를 직접 양육하지 않는 부모 일방과 자녀는 서로 만나거나 연락할 수 있...
3,이혼한 후에 자녀의 성과 본을 저의 성과 본으로 바꿀 수 있나요?,이혼 후 자녀의 성과 본을 자신의 성과 본으로 바꿀 수 있습니다. ◇ 법원 허가 자...
4,"중학생 딸아이가 학교 숙제로 유언장을 작성했는데, 이 유언장이 법적으로 효력 있는 ...",유언은 의사능력이 있는 17세(유언 적령)에 달한 사람이 할 수 있습니다. 따라서 ...
...,...,...
2270,"시청에 정기적으로 문구류를 납품하는 수의계약에 참여했는데, 수의계약의 계약상대자는 ...","수의계약대상자는 견적제출자의 견적가격과 계약이행능력 등에 따라 결정되며, 원칙적으로..."
2271,물품계약을 체결한 후 물가가 급격히 올라서 계약 당시의 금액으로는 수량을 맞추기 어...,아니요. 물품계약을 체결한 날부터 90일 이상 지난 후 입찰일을 기준일로 하여 품목...
2272,"계약을 체결한 후에는 계약 완료 전이라도 대금을 미리 받을 수 있다고 하던데, 얼마...",계약을 이행하기 전이라도 일정 요건에 해당하면 계약금액의 100분의 70을 초과하지...
2273,물품을 납품하기 직전에 천재지변 등 불가항력의 사유로 대형화재가 발생하여 해당 물품...,"계약상대자의 책임 없이 이행이 지체되는 경우에는 계약기간 연장신청을 할 수 있고, ..."


In [5]:
loader = DataFrameLoader(df, page_content_column="instruction")
document = loader.load()


In [6]:
# model_name = "Salesforce/SFR-Embedding-Mistral"
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(document, cached_embeddings)

In [7]:
retriever = vectorstore.as_retriever()

In [8]:
retrieved_docs = retriever.invoke("이혼이 취소될 수 있나요?")
retrieved_docs

[Document(page_content='이혼이 취소될 수 있나요?', metadata={'output': '재판상 이혼은 재판절차를 거쳐 이혼판결이 선고된 것이므로 취소될 수 없지만, 협의이혼은 부부간 자유로운 의사에 따른 합의에 기초하므로 사기 또는 강박(强迫)으로 인해 이혼의 의사표시를 했다면 이혼이 취소될 수 있습니다. 이혼을 취소하려면 관할 가정법원에 이혼취소소송을 제기해야 합니다. 이혼취소판결이 확정되면 그 이혼은 처음부터 없었던 것과 같아지므로 취소판결 전에 다른 일방이 재혼했다면 그 재혼은 중혼(重婚)이 됩니다. ◇ 중혼 ☞ 중혼은 법률상 혼인관계가 둘 이상 존재하는 위법한 상태로서 혼인의 취소사유가 됩니다. 중혼을 이유로 재혼(後婚)이 취소되면 전 배우자와만 법률상의 부부관계가 유지되고, 재혼 배우자와의 법률상 부부관계는 종료됩니다.'}),
 Document(page_content='이혼이 무효로 되는 경우가 있나요?', metadata={'output': '재판상 이혼은 재판절차를 거쳐 이혼판결이 선고된 것이므로 무효로 되지 않지만, 협의이혼은 부부 간 합의에 기초하므로 이혼에 관한 부부의 합의가 없다면 이혼이 무효로 될 수 있습니다. 예를 들어, 부부 일방 또는 쌍방이 모르는 사이에 누군가에 의해 이혼신고된 경우는 부부 사이에 이혼에 관한 합의가 없었으므로 이혼무효가 될 수 있습니다. 이혼을 무효로 하려면 관할 가정법원에 이혼무효소송을 제기해야 합니다. 이혼무효판결이 확정되면 그 이혼은 처음부터 없었던 것과 같아지므로 이전의 혼인은 중단 없이 계속된 것으로 됩니다.'}),
 Document(page_content='이혼해도 자녀를 만날 수 있나요?', metadata={'output': '이혼 후 자녀를 직접 양육하지 않는 부모 일방과 자녀는 서로 만나거나 연락할 수 있는 면접교섭권을 가집니다. 면접교섭권의 행사는 자녀의 복리를 우선적으로 고려해서 이루어져야 하므로, 자녀가 만남을 꺼려하는 등 자녀의 복리를 위해 필요한 경우에는 면접교섭이

In [9]:
model_id = "/home/crysis/projects/llama2"
peft_model_id = "/home/crysis/projects/LLM/05_finetuning/test/03_llama2_law/wandb/llama-law-model-0206"

config = PeftConfig.from_pretrained(peft_model_id)
bnb_confg = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id, quantization_config=bnb_confg, device_map={"":0})
model_pt = PeftModel.from_pretrained(model, peft_model_id)
pipe = pipeline("text-generation", model=model_pt, tokenizer=tokenizer, max_new_tokens=4096)
hf = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.27s/it]
/home/crysis/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/crysis/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameteriza

RuntimeError: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.model.embed_tokens.weight: copying a param with shape torch.Size([32002, 4096]) from checkpoint, the shape in current model is torch.Size([32000, 4096]).
	size mismatch for base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: copying a param with shape torch.Size([1024, 64]) from checkpoint, the shape in current model is torch.Size([4096, 64]).
	size mismatch for base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight: copying a param with shape torch.Size([1024, 64]) from checkpoint, the shape in current model is torch.Size([4096, 64]).
	size mismatch for base_model.model.model.layers.2.self_attn.v_proj.lora_B.default.weight: copying a param with shape torch.Size([1024, 64]) from checkpoint, the shape in current model is torch.Size([4096, 64]).
	size mismatch for base_model.model.model.layers.3.self_attn.v_proj.lora_B.default.weight: copying a pa

In [27]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

def gen(x):
    q = prompt % (x,)
    gened = model_pt.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=4096,
        early_stopping=True,
        do_sample=False,
    )
    print(q)
    return tokenizer.decode(gened[0]).replace(q, "")

In [28]:
gen("이혼 취소를 하고 싶은데 어떻게 해야해?")

Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: 이혼 취소를 하고 싶은데 어떻게 해야해? ### Response: 


'<s> 이혼 취소 신청은 당사자 간 합의로 협의해서 취소 하면 됩니다. ◇ 이혼 취소의 신청 ☞ 이혼의 취소는 당사자 간의 합의로 협의해서 취소 하면 됩니다. ☞ 따라서 일방 당사자의 의사표시나 강제와 같은 불법한 방법으로 이혼 시효 중인 경우에도 다른 일방이 반대하더라도 이혼 취소를 할 수 있습니다. ☞ 그러나 한쪽 당사자가 반대하고 다른 한쪽 당사자이라도 합의가 되면 이혼 취소는 성립됩니다.</s>'

In [21]:
template = """ 
        ### Instruction: {question}
        ### Response: """
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "이혼 취소를 하고 싶은데 어떻게 해야해?"

chain.invoke({"question": question})

/home/crysis/anaconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'이혼 취소의 신청은 당사자 간 이혼 취소의 합의가 있더라도, 일정 요건을 모두 갖춰야 합니다. ◇ 이혼 취소의 합의 ☞ 이혼의 취소는 당사자 간의 이혼 취소의 합의가 있더라도, ① 결혼관계가 일정기간 이상 존속되고 ② 이혼하지 않으면 당사자 간의 권리 의무 관계에 매우 큰 영향을 미치게 된다는 2가지 요건을 모두 갖추어야 합니다. ◇ 이혼 취소의 합의 시 간이혼 확인 ☞ 취소의 합의 시에 일정 기간의 유형 결혼관계가 존속한지 확인하려면 일정 기간의 유형 결혼관계를 존속하였는지를 하는 증명자료를 준비해야 합니다. ☞ 이를 위해 법원에 소장을 하여 법원이 그 사실을 확인한 후 합의의 합의 시에 그 사실을 감안할 수 있도록 해야 합니다. ◇ 이혼 취소의 합의 시 간이혼 관계행방향 확인 ☞ 취소의 합의 시에 결혼관계가 일정기간 이상 존속하였는지를 하는 증명자료를 준비할 때, 결혼 전과 이후 간 권리 변동사실 및 관계를 분석하여 그것으로 인해 결혼관계가 계속 유지되는 것으로 확인할 수 있도록 하려면 행방 형성을 감안할 수 있도록 해야 합니다.'

#### 불필요 코드

In [4]:
text_col = []
token_col = []
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")

for _, row in df.iterrows():
    instruction = str(row["instruction"])
    response = str(row["output"])
    
    text = "### Instruction:\n" + instruction + "\n### Response:\n" + response
    text_col.append(text)
    token = encoder.encode(text)
    token_col.append(len(token))

df.loc[:, "text"] = text_col
df.loc[:, "token"] = token_col
df.drop(['instruction', 'output'], axis=1, inplace=True)

In [ ]:
m = max(df['token'])
t = pd.Series(df['token']).sum()
print(f'Max Token: {m}\nTotal Token: {t}')